In [2]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *
from scipy.optimize import minimize_scalar
## get problem parameters and geometry
from netgen.geom2d import SplineGeometry
import sys
from IPython.display import clear_output
sys.path.insert(0,"../")
sys.path.insert(0,"../../")
import problems
import numpy as np
import params
import pickle
from time import time

In [6]:


problem = problems.problem2

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
ord = 4
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
# def mesher(geom, h):
#     if ".stp" in geom:
#         geo = OCCGeometry(geom)
#     else:
#         geo = pickle.load(open(geom, "rb"))

#     mesh = Mesh(geo.GenerateMesh(maxh=h))
#     return mesh
# mesh = mesher(geom, h)  
w,l= 3,15
n = 5



rect = SplineGeometry()
pnts = [(0,0), (l,0), (l,w), (0,w)]
p1 = rect.AppendPoint(*(0,0),hpref=1)
p2 = rect.AppendPoint(*(l,0),hpref=1)
p3 = rect.AppendPoint(*(l,w/4),hpref=1)
p4 = rect.AppendPoint(*(l,w),hpref=1)
p5 = rect.AppendPoint(*(0,w),hpref=1)
p6 = rect.AppendPoint(*(0,w/4),hpref=1)


curves = [[["line",p1,p2],"bottom"],
        [["line",p2,p3],"right_bot"],
        [["line",p3,p4],"right_top"],
        [["line",p4,p5],"top"],
        [["line",p5,p6],"left_top"],
        [["line",p6,p1],"left_bot"]]

wings_h = 0.03
top_h = 1
bot_h = 0.2
h = 1

rect.Append(curves[0][0], bc = curves[0][1], leftdomain=1, rightdomain=0, maxh=bot_h)
rect.Append(curves[1][0], bc = curves[1][1], leftdomain=1, rightdomain=0, maxh=wings_h) #0.03
rect.Append(curves[2][0], bc = curves[2][1], leftdomain=1, rightdomain=0, maxh=h) #0.03
rect.Append(curves[3][0], bc = curves[3][1], leftdomain=1, rightdomain=0, maxh=top_h) #0.05
rect.Append(curves[4][0], bc = curves[4][1], leftdomain=1, rightdomain=0,maxh=h) #0.03
rect.Append(curves[5][0], bc = curves[5][1], leftdomain=1, rightdomain=0, maxh=wings_h) #0.03


mesh = Mesh(rect.GenerateMesh(maxh=h)) #0.8
def Gel_energy_functional(F):
    
    g = 9.81
    J = Det(F)
    rho = 1.23e-6
    H = (J - phi0)*log(1-phi0/J)  + phi0 * chi*(1-phi0/J)
    # calculate C
    # min (G/2) * (3 lam**2) + KBTV * H(lam**3):  lam > 1
    # Hf = lambda lam: (lam-phi0)*np.log(1-phi0/lam) + phi0*chi*(1-phi0/lam)

    # fun = lambda lam: (G.Get()/2)*(3*lam**2) + KBTV*Hf(lam)
    # res = minimize_scalar(fun, bounds=(1, 10), method='bounded')
    # C = res.fun
    return 0.5*(G)* Trace(F.trans*F ) + H*KBTV #+ rho*g*u[1] - C


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord-3)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes, condense=True)






## static condensation

In [3]:

## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((0,0))

psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0
obs = CF(-y)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psik,psih = G_funs
    F = Id(2)+ Grad(u)
    if form == "Functional":
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx) # add the "bilinear" form of specific problem
        BF+= delta*v[1]*dx
        BF+= u[1]* vphi*dx
        BF+= -delta*exp(psih)*vphi*dx
        if ord==3:
            BF += -1e-6* delta * vphi*dx
        else: 
            BF += -1e-6* Grad(delta) * Grad(vphi)*dx
        # the right hand sideBF
        # BF += -alpha * force[2] * v[2] * dx # force is 0 
        BF += -(psik-psih)*v[1]*dx
        BF += -(obs+exp(psih))*vphi*dx
        return BF
    

Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
def Update_BF(BF,gfu,res):
    BF.AssembleLinearization(gfu.vec)
    invS = BF.mat.Inverse(freedofs=fes.FreeDofs(coupling=True))
    ext = IdentityMatrix() + BF.harmonic_extension
    extT = IdentityMatrix() + BF.harmonic_extension_trans
    invA =  ext @ invS @ extT + BF.inner_solve
    return invA, BF

def compute_damp(N_x, norm_N_0, t):
 
    tau = t**2*norm_N_0/2
    with TaskManager():
        norm_N_x = N_x.Norm()
    print("norm_N_x")
    print(np.sqrt(2*tau/norm_N_x))
    t_n = min(1,np.sqrt(2*tau/norm_N_x))
    return t_n

In [5]:
cd Proximal_Galerkin

[Errno 2] No such file or directory: 'Proximal_Galerkin'
/Users/sferra/Desktop/NonLInearElasticity/Proximal_Galerkin


/Users/sferra/Library/Python/3.13/lib/python/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [ ]:
max_PG_it = 100
max_iter_newton = 100
max_iter_qnewton = 100

adapt_damp = False
newton_damp = 0.1

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

reset = 4 # reset the inverse every reset iterations

initial_G = G_target*25
good_G = [initial_G]
good_gfu = [gfu.vec.data]
good_psih = [psih.vec.data]
G.Set(initial_G)
good_G.append(G.Get())
step = 0.5
not_Done = True


clear = False


while not_Done:
    if G.Get() == G_target:
        print("Target G achieved")
        not_Done = False
    try:
        print("Current load:", G.Get())
         #incremental softening
        for k in range(1,max_PG_it):
            print("PG it:",k)
            alpha.Set(1)
            psik.vec.data = psih.vec
            if k != 1:
                uk.vec.data = gfu.components[0].vec 

            G_funs = u,v,alpha,delta,psik, psih

            with TaskManager():
                BF = Assemble_BF(BF,G_funs,form)
                res = gfu.vec.CreateVector()
                w = gfu.vec.CreateVector()
            print("Solving nonlinear subproblem")
            for it in range(max_iter_qnewton):
                wh =  GridFunction(fesu)
                wh.Set(gfu.components[0])
                print("QN:", it)
                with TaskManager():
                    invA,BF = Update_BF(BF,gfu,res)
                for iter in range(max_iter_newton):
                    BF.Apply(gfu.vec,res)
                    if iter%reset == 0 and iter != 0:
                        with TaskManager():
                            invA,BF = Update_BF(BF,gfu,res)

                    with TaskManager():
                        w.data = newton_damp * invA * res
                        gfu.vec.data += -w
                        delt = InnerProduct(w,res)
            
                    if abs(delt) < tol_newton:
                        print("Tol achieved newton",delt)
                        break
                
                    # update psik
                psih.vec.data = psih.vec + gfu.components[1].vec
                wh_dif = Integrate((gfu.components[0]-wh)*(gfu.components[0]-wh),mesh)
                if wh_dif < tol_QN:
                    print("Tol achieved",wh_dif)
                    break
            dif = Integrate((uk-gfu.components[0])*(uk-gfu.components[0]),mesh)
            print("U_k difference", dif)
            if dif < tol_delta:
                print("Tol PG achieved", dif)
                break
        clear = True
        good_G.append(G.Get())
        good_gfu.append(gfu.vec.Copy())
        good_psih.append(psih.vec.Copy())
        clear_output()
        
        
    except:
        print("Exception occurred, reducing step size and reset window")
        gfu.vec.data = good_gfu[-1]
        psih.vec.data = good_psih[-1]
        load = good_G[-1]

        if reset != 1:
            reset -= 1
        step = step/2
        load =  load-step
        G.Set(load)
        print("load", load, "step", step, "reset", reset)
    
    if clear:
        print("Updating")
        print("Step", step)
        
        load = G.Get() - step
        if load < G_target:
            load = G_target
        G.Set(load)
        clear = False





Updating
Step 0.0078125
Current load: 0.9765625
PG it: 1
Solving nonlinear subproblem
QN: 0
Tol achieved newton 8.690194939589753e-07
QN: 1


In [61]:
pickle.dump([gfu,load,psih], open("temp_info.p", "wb"))


In [ ]:
cd /Users/sferra/Desktop/NonLInearElasticity/Proximal_Galerkin

In [ ]:
ls

In [64]:
# pickle.dump(gfu, open("gfu123.pkl", "wb+"))
# pickle.dump(psih, open("psi_h_mesh1.pkl", "wb+"))
# pickle.dump(mesh, open("mesh1.pkl", "wb+"))

In [ ]:
import

In [14]:
gfu=pickle.load(open("gfu123.pkl", "rb"))
psih = pickle.load(open("psi_h_mesh1.pkl", "rb"))
mesh =pickle.load(open("mesh1.pkl", "rb"))

In [15]:
## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord-3)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes, condense=True)


In [16]:
Draw(gfu.components[0], mesh, "u")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
import matplotlib.pyplot as plt
plt.plot(good_G)


In [19]:
obs = -y
gfu_tilde = GridFunction(fesu)
gfu_tilde.vec.data = gfu.components[0].vec
gfu_tilde.components[1].Set(exp(psih)+obs)
Draw(gfu_tilde, mesh)
u, v = fesu.TnT()
i = BilinearForm(fesu)
i += InnerProduct(u,v)*dx
i.Assemble()
L = LinearForm(fesu)
L += gfu_tilde*v*dx
L.Assemble()
gfu_t = GridFunction(fesu)
gfu_t.vec[:] = 0
gfu_t.vec.data += i.mat.Inverse(freedofs=fesu.FreeDofs())*L.vec
Draw(gfu_t, mesh, "gfu_t")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [9]:
Draw(exp(psih) + -y, mesh, "psi_h",min=0, max=1.5)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [20]:
Draw(gfu_t-gfu.components[0], mesh, "gfu_t-gfu")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

Check proximal point and projected optimization with constraints

In [70]:
import pandas as pd

df = pd.DataFrame()
columns = ["x", "y", "uh_y", "satisfy_constraint", "sum"]
data = []

In [71]:
uh = gfu.components[0]
for i,p in enumerate(mesh.vertices):
    temp = [0,0,0,0,0]
    if p.point[0] == 0:
        temp[0] = p.point[0]
        temp[1] = p.point[1]
        temp[2] = uh(mesh(*p.point))[1]
        temp[3] = (uh(mesh(*p.point))[1]+p.point[1]>0)
        temp[4] = uh(mesh(*p.point))[1]+p.point[1]
        data.append(temp)
df = pd.DataFrame(data, columns=columns)


In [ ]:
pd.set_option('display.max_rows', 500)
df